In [37]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline
import csv
import src.read_weather as rw
import psycopg2 as pg2
import sqlalchemy
import julian
import datetime
import s3fs

In [51]:
txt_path = 'data/weather/meta_data/ghcnd-stations.txt'
path_2000= 's3://plant-forecast/gz_files/2001.csv.gz'

In [38]:
weather_data=rw.read_weather_data(path_2000)

In [52]:
meta_data= rw.read_metadata_txt(txt_path)

In [53]:
meta_data.head()

,station_id,latitude,longitude,elevation,state
0,ACW00011604,17.1167,-61.7833,10.1,ST
1,ACW00011647,17.1333,-61.7833,19.2,ST
2,AE000041196,25.3330,55.5170,34.0,SHARJAH
3,AEM00041194,25.2550,55.3640,10.4,DUBAI
4,AEM00041217,24.4330,54.6510,26.8,ABU


EmptyDataError: No columns to parse from file

In [10]:
weather_data.head()

,station_id,measurement_date,measurement_type,measurement_flag


In [18]:
meta_data.head()

,station_id,latitude,longitude,elevation,state
0,ACW00011604,17.1167,-61.7833,10.1,ST
1,ACW00011647,17.1333,-61.7833,19.2,ST
2,AE000041196,25.3330,55.5170,34.0,SHARJAH
3,AEM00041194,25.2550,55.3640,10.4,DUBAI
4,AEM00041217,24.4330,54.6510,26.8,ABU


db_name = 'weather'
connection_string = "postgresql://localhost:5432/%s" % (db_name)
conn = sqlalchemy.create_engine(connection_string)

table_name = 'weather_data'
# The to_sql method defaults to bigint for integer types here, which are larger than we need. 
# This manually sets the datatypes of the columns we need to override
column_type_dict = {'measurement_flag': sqlalchemy.types.Integer}
# Writing all the data to the DB at once will cause this notebook to crash.
# We pass a large integer to the chunksize parameter to chunk the writing of records
weather_data.to_sql(table_name, conn, index_label='id', dtype=column_type_dict)

In [85]:
metadata_table_name = 'station_metadata'
station_metadata.to_sql(metadata_table_name, conn, index_label='id')

In [89]:
weather_type_dict = {'PRCP': 'Precipitation', 'SNOW': 'Snowfall', 'SNWD': 'Snow Depth', 
                     'TMAX': 'Maximum temperature', 'TMIN': 'Minimum temperature'}
weather_type_df = pd.DataFrame(weather_type_dict, columns=['weather_type', 'weather_description'])
description_table_name = 'weather_types'
weather_type_df.to_sql(description_table_name, conn, index_label='id')